### All days of the challange:

Here's what we're going to do today:

* [Take a first look at the data](#Take-a-first-look-at-the-data)
* [See how many missing data points we have](#See-how-many-missing-data-points-we-have)
* [Figure out why the data is missing](#Figure-out-why-the-data-is-missing)
* [Drop missing values](#Drop-missing-values)
* [Filling in missing values](#Filling-in-missing-values)

Let's get started!

# Take a first look at the data
________

The first thing we'll need to do is load in the libraries and datasets we'll be using. For today, I'll be using a dataset of events that occured in American Football games for demonstration, and you'll be using a dataset of building permits issued in San Francisco.

> **Important!** Make sure you run this cell yourself or the rest of your code won't work!

In [1]:
# libraries we'll use
import pandas as pd

# read in all our data NFL nfl1-> Play by Play 2009-2017 (v4), nfl2 -> NFL Play by Play 2009-2016 (v3)
nfl1 = pd.read_csv("NFL Play by Play 2009-2017 (v4).csv")
nfl2 = pd.read_csv("NFL Play by Play 2009-2016 (v3).csv")



C:\Users\Mahmoud Elshafei\AppData\Local\Temp\ipykernel_7972\1444739442.py:5: DtypeWarning: Columns (25,51) have mixed types. Specify dtype option on import or set low_memory=False.
  nfl1 = pd.read_csv("NFL Play by Play 2009-2017 (v4).csv")
C:\Users\Mahmoud Elshafei\AppData\Local\Temp\ipykernel_7972\1444739442.py:6: DtypeWarning: Columns (25,51) have mixed types. Specify dtype option on import or set low_memory=False.
  nfl2 = pd.read_csv("NFL Play by Play 2009-2016 (v3).csv")


The first thing I do when I get a new dataset is take a look at some of it. This lets me see that it all read in correctly and get an idea of what's going on with the data. In this case, I'm looking to see if I see any missing values, which will be reprsented with `NaN` or `None`.

In [2]:
# look at a few rows of the nfl1_data file. I can see a handful of missing data already!

nfl1.head()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2009-09-10,2009091000,1,1,NaN,15:00,15,3600.0,0.0,TEN,...,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009
1,2009-09-10,2009091000,1,1,1.0,14:53,15,3593.0,7.0,PIT,...,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,2009-09-10,2009091000,1,1,2.0,14:16,15,3556.0,37.0,PIT,...,NaN,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,NaN,NaN,2009
3,2009-09-10,2009091000,1,1,3.0,13:35,14,3515.0,41.0,PIT,...,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2009-09-10,2009091000,1,1,4.0,13:27,14,3507.0,8.0,PIT,...,NaN,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,NaN,NaN,2009


Yep, it looks like there's some missing values. What about in the nfl2 dataset?

In [3]:
# your turn! Look at a couple of rows from the nfl2 dataset. Do you notice any missing data?

# your code goes here :)
nfl2.head(2)

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2009-09-10,2009091000,1,1,NaN,15:00,15,3600.0,0.0,TEN,...,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009
1,2009-09-10,2009091000,1,1,1.0,14:53,15,3593.0,7.0,PIT,...,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009


# See how many missing data points we have
___

Ok, now we know that we do have some missing values. Let's see how many we have in each column. 

In [4]:
# get the number of missing data points per column
missing_data = nfl1.isnull().sum()

# look at the # of missing points in the first ten columns

print(missing_data.head(10))

Date                0
GameID              0
Drive               0
qtr                 0
down            61154
time              224
TimeUnder           0
TimeSecs          224
PlayTimeDiff      444
SideofField       528
dtype: int64


That seems like a lot! It might be helpful to see what percentage of the values in our dataset were missing to give us a better sense of the scale of this problem:

In [7]:
# how many total missing values do we have?
tot1 = nfl1.isnull().values.sum()
print(tot1)

# percent of data that is missing
percentage = (tot1 / nfl1.size) * 100
print(percentage)

11505187
27.66722370547874


Wow, almost a quarter of the cells in this dataset are empty! In the next step, we're going to take a closer look at some of the columns with missing values and try to figure out what might be going on with them.

In [8]:
# your turn! Find out what percent of the nfl2_ dataset is missing
tot2 = nfl2.isnull().values.sum()

percentage2 = (tot2 / nfl2.size) * 100
print(percentage2)

27.652267428200588


# Figure out why the data is missing
____
 
This is the point at which we get into the part of data science that I like to call "data intution", by which I mean "really looking at your data and trying to figure out why it is the way it is and how that will affect your analysis". It can be a frustrating part of data science, especially if you're newer to the field and don't have a lot of experience. For dealing with missing values, you'll need to use your intution to figure out why the value is missing. One of the most important question you can ask yourself to help figure this out is this:

> **Is this value missing becuase it wasn't recorded or becuase it dosen't exist?**

If a value is missing becuase it doens't exist (like the height of the oldest child of someone who doesn't have any children) then it doesn't make sense to try and guess what it might be. These values you probalby do want to keep as NaN. On the other hand, if a value is missing becuase it wasn't recorded, then you can try to guess what it might have been based on the other values in that column and row. (This is called "imputation" and we'll learn how to do it next! :)

Let's work through an example. Looking at the number of missing values in the nfl_data dataframe, I notice that the column `TimesSec` has a lot of missing values in it: 

In [9]:
# look at the # of missing points in the first ten columns
missing_data.head(10)

Date                0
GameID              0
Drive               0
qtr                 0
down            61154
time              224
TimeUnder           0
TimeSecs          224
PlayTimeDiff      444
SideofField       528
dtype: int64

By looking at [the documentation](https://www.kaggle.com/maxhorowitz/nflplaybyplay2009to2016), I can see that this column has information on the number of seconds left in the game when the play was made. This means that these values are probably missing because they were not recorded, rather than because they don't exist. So, it would make sense for us to try and guess what they should be rather than just leaving them as NA's.

On the other hand, there are other fields, like `PenalizedTeam` that also have lot of missing fields. In this case, though, the field is missing because if there was no penalty then it doesn't make sense to say *which* team was penalized. For this column, it would make more sense to either leave it empty or to add a third value like "neither" and use that to replace the NA's.

> **Tip:** This is a great place to read over the dataset documentation if you haven't already! If you're working with a dataset that you've gotten from another person, you can also try reaching out to them to get more information.

If you're doing very careful data analysis, this is the point at which you'd look at each column individually to figure out the best strategy for filling those missing values. For the rest of this notebook, we'll cover some "quick and dirty" techniques that can help you with missing values but will probably also end up removing some useful information or adding some noise to your data.

## Your turn!

* Look at the columns `Street Number Suffix` and `Zipcode` from the `sf_permits` datasets. Both of these contain missing values. Which, if either, of these are missing because they don't exist? Which, if either, are missing because they weren't recorded?

# Drop missing values
___

If you're in a hurry or don't have a reason to figure out why your values are missing, one option you have is to just remove any rows or columns that contain missing values. (Note: I don't generally recommend this approch for important projects! It's usually worth it to take the time to go through your data and really look at all the columns with missing values one-by-one to really get to know your dataset.)  

If you're sure you want to drop rows with missing values, pandas does have a handy function, `dropna()` to help you do this. Let's try it out on our NFL dataset!

In [12]:
# remove all the rows that contain a missing value
nfl1.dropna()
nfl2.dropna()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season


Oh dear, it looks like that's removed all our data! 😱 This is because every row in our dataset had at least one missing value. We might have better luck removing all the *columns* that have at least one missing value instead.

In [15]:
# remove all columns with at least one missing value
removed_columns1 = nfl1.dropna(axis=1)
removed_columns2 = nfl2.dropna(axis=1)

removed_columns1
removed_columns2

,Date,GameID,Drive,qtr,TimeUnder,ydstogo,ydsnet,PlayAttempted,Yards.Gained,sp,...,AwayTeam,Timeout_Indicator,posteam_timeouts_pre,HomeTimeouts_Remaining_Pre,AwayTimeouts_Remaining_Pre,HomeTimeouts_Remaining_Post,AwayTimeouts_Remaining_Post,ExPoint_Prob,TwoPoint_Prob,Season
0,2009-09-10,2009091000,1,1,15,0,0,1,39,0,...,TEN,0,3,3,3,3,3,0.000000,0.0,2009
1,2009-09-10,2009091000,1,1,15,10,5,1,5,0,...,TEN,0,3,3,3,3,3,0.000000,0.0,2009
2,2009-09-10,2009091000,1,1,15,5,2,1,-3,0,...,TEN,0,3,3,3,3,3,0.000000,0.0,2009
3,2009-09-10,2009091000,1,1,14,8,2,1,0,0,...,TEN,0,3,3,3,3,3,0.000000,0.0,2009
4,2009-09-10,2009091000,1,1,14,8,2,1,0,0,...,TEN,0,3,3,3,3,3,0.000000,0.0,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362442,2017-01-01,2017010102,20,4,1,10,35,1,35,1,...,GB,0,0,0,0,0,0,0.000000,0.0,2016
362443,2017-01-01,2017010102,20,4,1,0,35,1,0,1,...,GB,0,0,0,0,0,0,0.931115,0.0,2016
362444,2017-01-01,2017010102,21,4,1,0,0,1,0,0,...,GB,0,0,0,0,0,0,0.000000,0.0,2016
362445,2017-01-01,2017010102,21,4,1,10,-1,1,-1,0,...,GB,0,0,0,0,0,0,0.000000,0.0,2016


In [21]:
# just how much data did we lose?
lost1 = nfl1.shape[1] - removed_columns1.shape[1]
lost2 = nfl2.shape[1] - removed_columns2.shape[1]

print(lost1, lost2)

65 65


We've lost quite a bit of data, but at this point we have successfully removed all the `NaN`'s from our data. 

In [22]:
# Your turn! Try removing all the rows from the nfl2 dataset that contain missing values. How many are left?
nfl2.dropna()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season


In [23]:
# Now try removing all the columns with empty values. Now how much of your data is left?
nfl2.dropna(axis=1)

,Date,GameID,Drive,qtr,TimeUnder,ydstogo,ydsnet,PlayAttempted,Yards.Gained,sp,...,AwayTeam,Timeout_Indicator,posteam_timeouts_pre,HomeTimeouts_Remaining_Pre,AwayTimeouts_Remaining_Pre,HomeTimeouts_Remaining_Post,AwayTimeouts_Remaining_Post,ExPoint_Prob,TwoPoint_Prob,Season
0,2009-09-10,2009091000,1,1,15,0,0,1,39,0,...,TEN,0,3,3,3,3,3,0.000000,0.0,2009
1,2009-09-10,2009091000,1,1,15,10,5,1,5,0,...,TEN,0,3,3,3,3,3,0.000000,0.0,2009
2,2009-09-10,2009091000,1,1,15,5,2,1,-3,0,...,TEN,0,3,3,3,3,3,0.000000,0.0,2009
3,2009-09-10,2009091000,1,1,14,8,2,1,0,0,...,TEN,0,3,3,3,3,3,0.000000,0.0,2009
4,2009-09-10,2009091000,1,1,14,8,2,1,0,0,...,TEN,0,3,3,3,3,3,0.000000,0.0,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362442,2017-01-01,2017010102,20,4,1,10,35,1,35,1,...,GB,0,0,0,0,0,0,0.000000,0.0,2016
362443,2017-01-01,2017010102,20,4,1,0,35,1,0,1,...,GB,0,0,0,0,0,0,0.931115,0.0,2016
362444,2017-01-01,2017010102,21,4,1,0,0,1,0,0,...,GB,0,0,0,0,0,0,0.000000,0.0,2016
362445,2017-01-01,2017010102,21,4,1,10,-1,1,-1,0,...,GB,0,0,0,0,0,0,0.000000,0.0,2016


# Filling in missing values automatically
_____

Another option is to try and fill in the missing values. For this next bit, I'm getting a small sub-section of the NFL data so that it will print well.

In [24]:
# get a small subset of the NFL dataset
sub = nfl2.sample(n=10)
sub

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
113997,2011-11-13,2011111305,7,2,2.0,07:06,8,2226.0,34.0,KC,...,NaN,0.225209,0.774791,0.240560,0.759440,0.225209,0.015351,NaN,NaN,2011
72542,2010-11-21,2010112113,22,4,NaN,02:48,3,168.0,3.0,PHI,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,2010
212682,2013-12-01,2013120106,14,3,3.0,14:17,15,1757.0,22.0,PHI,...,NaN,0.838311,0.161689,0.848498,0.151502,0.838311,0.010187,NaN,NaN,2013
359544,2016-12-26,2016122600,12,3,1.0,15:00,15,1800.0,0.0,DET,...,NaN,0.482654,0.517346,0.507258,0.492742,0.517346,-0.024604,NaN,NaN,2016
289671,2015-10-29,2015102900,2,1,2.0,07:32,8,3152.0,12.0,MIA,...,0.953041,0.719594,0.280406,0.639452,0.360548,0.280406,0.080142,0.047205,0.032937,2015
66751,2010-11-07,2010110709,9,2,NaN,01:11,2,1871.0,10.0,PHI,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,2010
102433,2011-10-09,2011100910,5,1,1.0,04:07,5,2947.0,35.0,DEN,...,NaN,0.480965,0.519035,0.487952,0.512048,0.519035,-0.006987,NaN,NaN,2011
215001,2013-12-08,2013120800,15,2,2.0,00:04,1,1804.0,16.0,MIN,...,NaN,0.703171,0.296829,0.703171,0.296829,0.703171,0.000000,NaN,NaN,2013
38356,2009-12-20,2009122000,22,4,3.0,11:14,12,674.0,42.0,CHI,...,-4.003826,0.990638,0.009362,0.995644,0.004356,0.009362,-0.005006,0.008608,-0.013614,2009
349731,2016-12-04,2016120403,20,4,NaN,03:46,4,226.0,6.0,PHI,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,2016


We can use the Panda's fillna() function to fill in missing values in a dataframe for us. One option we have is to specify what we want the `NaN` values to be replaced with. Here, I'm saying that I would like to replace all the `NaN` values with 0.

In [25]:
# replace all NA's with 0
sub.fillna(0)

C:\Users\Mahmoud Elshafei\AppData\Local\Temp\ipykernel_7972\4041491986.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sub.fillna(0)


,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
113997,2011-11-13,2011111305,7,2,2.0,07:06,8,2226.0,34.0,KC,...,0.000000,0.225209,0.774791,0.240560,0.759440,0.225209,0.015351,0.000000,0.000000,2011
72542,2010-11-21,2010112113,22,4,0.0,02:48,3,168.0,3.0,PHI,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2010
212682,2013-12-01,2013120106,14,3,3.0,14:17,15,1757.0,22.0,PHI,...,0.000000,0.838311,0.161689,0.848498,0.151502,0.838311,0.010187,0.000000,0.000000,2013
359544,2016-12-26,2016122600,12,3,1.0,15:00,15,1800.0,0.0,DET,...,0.000000,0.482654,0.517346,0.507258,0.492742,0.517346,-0.024604,0.000000,0.000000,2016
289671,2015-10-29,2015102900,2,1,2.0,07:32,8,3152.0,12.0,MIA,...,0.953041,0.719594,0.280406,0.639452,0.360548,0.280406,0.080142,0.047205,0.032937,2015
66751,2010-11-07,2010110709,9,2,0.0,01:11,2,1871.0,10.0,PHI,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2010
102433,2011-10-09,2011100910,5,1,1.0,04:07,5,2947.0,35.0,DEN,...,0.000000,0.480965,0.519035,0.487952,0.512048,0.519035,-0.006987,0.000000,0.000000,2011
215001,2013-12-08,2013120800,15,2,2.0,00:04,1,1804.0,16.0,MIN,...,0.000000,0.703171,0.296829,0.703171,0.296829,0.703171,0.000000,0.000000,0.000000,2013
38356,2009-12-20,2009122000,22,4,3.0,11:14,12,674.0,42.0,CHI,...,-4.003826,0.990638,0.009362,0.995644,0.004356,0.009362,-0.005006,0.008608,-0.013614,2009
349731,2016-12-04,2016120403,20,4,0.0,03:46,4,226.0,6.0,PHI,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2016


I could also be a bit more savvy and replace missing values with whatever value comes directly after it in the same column. (This makes a lot of sense for datasets where the observations have some sort of logical order to them.)

In [26]:
# replace all NA's the value that comes directly after it in the same column, 
# then replace all the reamining na's with 0
sub.fillna(method='bfill')
sub.fillna(0)

C:\Users\Mahmoud Elshafei\AppData\Local\Temp\ipykernel_7972\3507837211.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  sub.fillna(method='bfill')
C:\Users\Mahmoud Elshafei\AppData\Local\Temp\ipykernel_7972\3507837211.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sub.fillna(method='bfill')
C:\Users\Mahmoud Elshafei\AppData\Local\Temp\ipykernel_7972\3507837211.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sub.fillna(0)


,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
113997,2011-11-13,2011111305,7,2,2.0,07:06,8,2226.0,34.0,KC,...,0.000000,0.225209,0.774791,0.240560,0.759440,0.225209,0.015351,0.000000,0.000000,2011
72542,2010-11-21,2010112113,22,4,0.0,02:48,3,168.0,3.0,PHI,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2010
212682,2013-12-01,2013120106,14,3,3.0,14:17,15,1757.0,22.0,PHI,...,0.000000,0.838311,0.161689,0.848498,0.151502,0.838311,0.010187,0.000000,0.000000,2013
359544,2016-12-26,2016122600,12,3,1.0,15:00,15,1800.0,0.0,DET,...,0.000000,0.482654,0.517346,0.507258,0.492742,0.517346,-0.024604,0.000000,0.000000,2016
289671,2015-10-29,2015102900,2,1,2.0,07:32,8,3152.0,12.0,MIA,...,0.953041,0.719594,0.280406,0.639452,0.360548,0.280406,0.080142,0.047205,0.032937,2015
66751,2010-11-07,2010110709,9,2,0.0,01:11,2,1871.0,10.0,PHI,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2010
102433,2011-10-09,2011100910,5,1,1.0,04:07,5,2947.0,35.0,DEN,...,0.000000,0.480965,0.519035,0.487952,0.512048,0.519035,-0.006987,0.000000,0.000000,2011
215001,2013-12-08,2013120800,15,2,2.0,00:04,1,1804.0,16.0,MIN,...,0.000000,0.703171,0.296829,0.703171,0.296829,0.703171,0.000000,0.000000,0.000000,2013
38356,2009-12-20,2009122000,22,4,3.0,11:14,12,674.0,42.0,CHI,...,-4.003826,0.990638,0.009362,0.995644,0.004356,0.009362,-0.005006,0.008608,-0.013614,2009
349731,2016-12-04,2016120403,20,4,0.0,03:46,4,226.0,6.0,PHI,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2016


Filling in missing values is also known as "imputation", and you can find more exercises on it [in this lesson, also linked under the "More practice!" section](https://www.kaggle.com/dansbecker/handling-missing-values). First, however, why don't you try replacing some of the missing values in the sf_permit dataset?